In [239]:
## Importing requireed libraries

import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [240]:
text = 'Pizza, the delectable and iconic dish that has transcended borders and captivated taste buds worldwide, is a testament to the extraordinary fusion of flavors, creativity, and cultural significance. Originating from the sun-kissed lands of Italy, pizza has evolved into an art form that unites people from diverse backgrounds in a shared love for its mouthwatering combinations. Its history stretches back centuries, with roots tracing back to ancient civilizations like the Greeks, Romans, and Egyptians, who all had their versions of flatbreads adorned with various ingredients. However, it was the vibrant city of Naples, Italy, that birthed the pizza we know and adore today.'

Text Preprocessing

In [241]:
# Word tokeinzer

tokenizer = Tokenizer()
words_ = tokenizer.fit_on_texts([text])

In [242]:
# Indexing the words

seq_text = tokenizer.texts_to_sequences([text])[0]
seq_text

[4,
 1,
 14,
 2,
 15,
 16,
 5,
 6,
 17,
 18,
 2,
 19,
 20,
 21,
 22,
 23,
 7,
 24,
 8,
 1,
 25,
 26,
 3,
 27,
 28,
 2,
 29,
 30,
 31,
 9,
 1,
 32,
 33,
 34,
 3,
 10,
 4,
 6,
 35,
 36,
 37,
 38,
 39,
 5,
 40,
 41,
 9,
 42,
 43,
 44,
 7,
 45,
 46,
 47,
 11,
 48,
 49,
 11,
 50,
 51,
 12,
 52,
 13,
 53,
 54,
 12,
 8,
 55,
 56,
 57,
 1,
 58,
 59,
 2,
 60,
 61,
 62,
 63,
 64,
 65,
 3,
 66,
 67,
 13,
 68,
 69,
 70,
 71,
 72,
 1,
 73,
 74,
 3,
 75,
 10,
 5,
 76,
 1,
 4,
 77,
 78,
 2,
 79,
 80]

In [243]:
# checking the vocab size

vocab_size = len(tokenizer.word_index)+1   ## We add 1 because it does not count from zero
print('Size of vocab i.e. No. of words=', vocab_size)

Size of vocab i.e. No. of words= 81


Next step is making sequence of words

In [244]:
# Creating a list

sequences = []
for i in range (1, len(seq_text)):
  sequence = seq_text[i-1 : i+1]
  sequences.append(sequence)

print('Total No. of Sequence=', len(sequences))


Total No. of Sequence= 103


In [245]:
sequences

[[4, 1],
 [1, 14],
 [14, 2],
 [2, 15],
 [15, 16],
 [16, 5],
 [5, 6],
 [6, 17],
 [17, 18],
 [18, 2],
 [2, 19],
 [19, 20],
 [20, 21],
 [21, 22],
 [22, 23],
 [23, 7],
 [7, 24],
 [24, 8],
 [8, 1],
 [1, 25],
 [25, 26],
 [26, 3],
 [3, 27],
 [27, 28],
 [28, 2],
 [2, 29],
 [29, 30],
 [30, 31],
 [31, 9],
 [9, 1],
 [1, 32],
 [32, 33],
 [33, 34],
 [34, 3],
 [3, 10],
 [10, 4],
 [4, 6],
 [6, 35],
 [35, 36],
 [36, 37],
 [37, 38],
 [38, 39],
 [39, 5],
 [5, 40],
 [40, 41],
 [41, 9],
 [9, 42],
 [42, 43],
 [43, 44],
 [44, 7],
 [7, 45],
 [45, 46],
 [46, 47],
 [47, 11],
 [11, 48],
 [48, 49],
 [49, 11],
 [11, 50],
 [50, 51],
 [51, 12],
 [12, 52],
 [52, 13],
 [13, 53],
 [53, 54],
 [54, 12],
 [12, 8],
 [8, 55],
 [55, 56],
 [56, 57],
 [57, 1],
 [1, 58],
 [58, 59],
 [59, 2],
 [2, 60],
 [60, 61],
 [61, 62],
 [62, 63],
 [63, 64],
 [64, 65],
 [65, 3],
 [3, 66],
 [66, 67],
 [67, 13],
 [13, 68],
 [68, 69],
 [69, 70],
 [70, 71],
 [71, 72],
 [72, 1],
 [1, 73],
 [73, 74],
 [74, 3],
 [3, 75],
 [75, 10],
 [10, 5],
 [5, 

In [246]:
max_sequence_len = max([len(seq) for seq in sequences])
print(max_sequence_len)

#sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

2


Let's start making the model

In [247]:
# Dividing the data into X and Y
# X: 1st column of seq
# Y: 2nd column of seq

sequences = np.array(sequences)
x, y = sequences[:,0], sequences[:,1]

In [248]:
x.shape

(103,)

In [249]:
x

array([ 4,  1, 14,  2, 15, 16,  5,  6, 17, 18,  2, 19, 20, 21, 22, 23,  7,
       24,  8,  1, 25, 26,  3, 27, 28,  2, 29, 30, 31,  9,  1, 32, 33, 34,
        3, 10,  4,  6, 35, 36, 37, 38, 39,  5, 40, 41,  9, 42, 43, 44,  7,
       45, 46, 47, 11, 48, 49, 11, 50, 51, 12, 52, 13, 53, 54, 12,  8, 55,
       56, 57,  1, 58, 59,  2, 60, 61, 62, 63, 64, 65,  3, 66, 67, 13, 68,
       69, 70, 71, 72,  1, 73, 74,  3, 75, 10,  5, 76,  1,  4, 77, 78,  2,
       79])

In [250]:
# One hot encoding Y-variable

y = to_categorical(y, num_classes=vocab_size)

In [251]:
y.shape

(103, 81)

In [252]:
# Defining the Neural Network

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1, 10)             810       
                                                                 
 lstm_8 (LSTM)               (None, 100)               44400     
                                                                 
 dense_8 (Dense)             (None, 81)                8181      
                                                                 
Total params: 53391 (208.56 KB)
Trainable params: 53391 (208.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [253]:
# Compiling the network

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [254]:
# Fitting our neural network model to data

model.fit(x, y, epochs=400)

Epoch 1/400
4/4 [==============================] - 2s 6ms/step - loss: 4.3949 - accuracy: 0.0194
Epoch 2/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3928 - accuracy: 0.0680
Epoch 3/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3912 - accuracy: 0.0874
Epoch 4/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3897 - accuracy: 0.0874
Epoch 5/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3881 - accuracy: 0.0874
Epoch 6/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3865 - accuracy: 0.0971
Epoch 7/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3850 - accuracy: 0.0971
Epoch 8/400
4/4 [==============================] - 0s 4ms/step - loss: 4.3832 - accuracy: 0.0971
Epoch 9/400
4/4 [==============================] - 0s 4ms/step - loss: 4.3814 - accuracy: 0.0971
Epoch 10/400
4/4 [==============================] - 0s 3ms/step - loss: 4.3792 - accuracy: 0.0971
Epoch 11/400
4/4 [===========

In [255]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [257]:
seed_text = "Pizza has "
next_words = 5
max_sequence_len = len(sequences)
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 18ms/step
Next predicted words: Pizza has  has evolved into an art
